In [2]:
import json

# Function to read your JSON file
def read_json_file(input_file):
    with open(input_file, 'r') as f:
        return json.load(f)

# Function to convert the input JSON to GeoJSON format
def convert_to_geojson(data):
    geojson = {
        "type": "FeatureCollection",  # Root level type
        "features": []
    }

    for item in data:
        # Create the Feature object
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",  # Assuming coordinates are for a point. Adjust if needed
                "coordinates": item["features"]["geometry"]["coordinates"]
            },
            "properties": item["features"]["properties"]
        }

        # Append the feature to the FeatureCollection
        geojson["features"].append(feature)

    return geojson

# Function to write the output GeoJSON file
def write_geojson_file(output_file, geojson_data):
    with open(output_file, 'w') as f:
        json.dump(geojson_data, f, indent=2)
    print(f"GeoJSON saved to {output_file}")

# Main function to process the file
def process_file(input_file, output_file):
    # Step 1: Read the input JSON file
    data = read_json_file(input_file)
    
    # Step 2: Convert it to GeoJSON format
    geojson_data = convert_to_geojson(data)
    
    # Step 3: Write the output GeoJSON to a file
    write_geojson_file(output_file, geojson_data)

# Input and output file paths
input_file = '../DataSets/Kalavryton_from_mongo.json' 
output_file = 'output.geojson'

# Run the script
process_file(input_file, output_file)


GeoJSON saved to output.geojson
